**This notebook is in Python 2.7**

In [3]:
import pandas as pd
from pattern.en import parsetree

In [4]:
df = pd.read_csv("https://s3.amazonaws.com/assets.datacamp.com/blog_assets/fake_or_real_news.csv")
print(df.shape)
print("Number of fakes: ",df[df['label'].str.contains("FAKE")].shape[0])
df = df[['title','text','label']]
df.head()

(6335, 4)
('Number of fakes: ', 3164)


,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


**Parse headlines into linguistic parse trees**

In [40]:
df['title_chunks'] = df[['title']].apply(lambda x: [chunk.type for chunk in parsetree(x['title'])[0].chunks], axis=1)
df.head()

0                                            [NP, VP, NP]
1                                [NP, VP, NP, PP, NP, NP]
2                            [NP, VP, NP, PP, NP, PP, NP]
3            [NP, PP, NP, VP, PP, NP, PP, NP, NP, VP, NP]
4                                        [NP, PP, NP, NP]
5                                                [NP, NP]
6                [NP, PP, NP, VP, NP, PP, NP, VP, NP, PP]
7                                                [NP, PP]
8                                    [NP, NP, PP, NP, NP]
9                            [NP, VP, NP, PP, NP, PP, NP]
10                   [PP, NP, PP, NP, NP, PP, NP, VP, NP]
11                                         [NP, VP, ADJP]
12                                           [NP, NP, NP]
13                                   [NP, VP, NP, PP, NP]
14                                 [NP, VP, PP, NP, ADVP]
15                                         [NP, VP, ADVP]
16                                                 [ADJP]
17            